---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# PULL DATA

In [1]:
import pandas as pd
from pandas_datareader import DataReader as pdr

# Pull data from FRED
files = ["DGS" + x for x in ["1MO", "3MO", "1", "2", "3", "5", "10", "20", "30"]]
df = pdr(files, "fred", start='2000-01-01') / 100
df.index.name = "date"
df = df.reset_index()

# Keep 1st observations of each month
df["month"] = df.date.dt.to_period("M").astype(str)
df = df.groupby("month").first()

# Clean-up data
df = df.drop(columns=["date"])
df = df.dropna(subset=["DGS3MO", "DGS30"])
df.columns = [1 / 12, 1 / 4, 1, 2, 3, 5, 10, 20, 30]
df = df.stack()
df = df.reset_index()
df.columns = ["date", "years to maturity", "yield"]
df.head()


,date,years to maturity,yield
0,2000-01,0.25,0.0548
1,2000-01,1.00,0.0609
2,2000-01,2.00,0.0638
3,2000-01,3.00,0.0642
4,2000-01,5.00,0.0650


# FIGURE

In [2]:
import plotly.express as px
fig = px.line(
    df,
    x="years to maturity",
    y="yield",
    animation_frame="date",
    hover_data={"date": True, "years to maturity": True, "yield": True}
)
fig.layout.xaxis["title"] = "Years to Maturity"
fig.layout.yaxis["title"] = "Yield"
fig.update_yaxes(tickformat=".1%", range=[0, df["yield"].max() + 0.001])
fig.update_xaxes(tickvals=[1, 3, 5, 10, 20, 30], range=[0, 30])
fig.for_each_trace(lambda t: t.update(mode='lines+markers', marker=dict(size=10)))
for fr in fig.frames:
    for d in fr.data:
        d.update(mode='markers+lines', marker=dict(size=10))
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 200
fig.show()